https://huggingface.co/docs/transformers/tasks/sequence_classification

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!pip install transformers datasets torch

Loading the model:

In [26]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    pipeline
)
from datasets import load_dataset
import torch

In [6]:
model_name = "roberta-base-openai-detector"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Loading the dataset:

In [41]:
ott_dataset = load_dataset(
    "csv",
    data_files= "/content/drive/MyDrive/data/ott_et_al_processed.csv",
    delimiter=";"
)

In [40]:
ott_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'truthful'],
        num_rows: 1600
    })
})

In [31]:
classifier = pipeline("text-classification", model=model_name)
classifier(ott_dataset['text'])

Some weights of the model checkpoint at roberta-base-openai-detector were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyError: ignored

In [25]:
with torch.no_grad():
  logits = model(**inputs).logits

TypeError: ignored